In [4]:
import sys 
from uvars import dataops_dir 
sys.path.append(dataops_dir)

from glob import glob

In [28]:
from dataops.transec import transect_augment_data
from dataops.rops import reproject_files_seq, list2txt,gdalbuildvrt,clip_raster_by_vector_bbox
from os.path import join

In [29]:
pdem_fn= "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/pdem/pdem.vrt"
s1_fn=  "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/s1/s1.vrt"
s2_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/s2/s2.vrt"
esawc_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/esawc/esawc.vrt"
ldem_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/ldem/ldem.vrt"
edem_wgs_fn = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRTV2/edem_wgs/edem_wgs.vrt"

cutline_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/cutline/"
bbox_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/bbox/"
exp_dir = "/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment/"
reproj_dir = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj"
tiff_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_OverlapPatchesLiDAR/*.tif")
bbox_files = glob(f"{bbox_dir}/*DTM_NP*.gpkg")
bbox_fn = bbox_files[0]

In [30]:
txt = join(reproj_dir,'transects.txt')
vrt = join(reproj_dir,'transects.vrt')
epsgcode = 4749#4326
reprojected_files = reproject_files_seq(tiff_files, reproj_dir,epsgcode)
list2txt(txt, files=reprojected_files)
gdalbuildvrt(vrt, txt, epsgcode)

Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0007_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0018_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0019_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0020_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0215_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0384_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0388_reproj.tif already exists. Skipping reprojection.
Output file /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/RNG_reproj/DTM_NP_T-0389_reproj.tif already

In [41]:
import os 
import pandas as pd 
import rasterio

In [50]:
def create_zdiff(dsm_path, dtm_path, dif_path, ndv=-9999.0):
    """
    Computes the difference (DSM - DTM) and saves it to the specified output path.
    
    Parameters:
        dsm_path (str): Path to the DSM raster file.
        dtm_path (str): Path to the DTM raster file.
        dif_path (str): Path to save the difference raster (DSM - DTM).
        ndv (float): Nodata value for the output raster (default is -9999.0).
        
    Returns:
        None
    """
    # Check if the output file already exists
    if os.path.exists(dif_path):
        print(f"Output file {dif_path} already exists. Skipping computation.")
        return

    # Open the input rasters
    with rasterio.open(dsm_path) as dsm_src, rasterio.open(dtm_path) as dtm_src:
        # Validate that the rasters are compatible
        
        # reshape 
        if dsm_src.crs != dtm_src.crs:
            raise ValueError("Input rasters do not have the same CRS.")
        
        if dsm_src.shape != dtm_src.shape:
            raise ValueError("Input rasters do not have the same dimensions.")
        # assing 
        #if dsm_src.transform != dtm_src.transform:
            #raise ValueError("Input rasters do not have the same transform.")

        # Read the raster data
        dsm = dsm_src.read(1)  # Read the first band of DSM
        dtm = dtm_src.read(1)  # Read the first band of DTM

        # Compute the difference (DSM - DTM)
        dif = dsm - dtm

        # Apply nodata mask: retain only valid DTM values
        valid_mask = (dtm > -99.) & (dtm < 10000)  # Define valid DTM range
        dif[~valid_mask] = ndv  # Set invalid areas to nodata

        # Copy metadata from one of the input rasters and update nodata value
        out_meta = dsm_src.meta
        out_meta.update({
            'nodata': ndv,
            'dtype': 'float32'  # Ensure output dtype is float32 for better GIS compatibility
        })

    # Write the result to the output path
    with rasterio.open(dif_path, "w", **out_meta) as dst:
        dst.write(dif.astype('float32'), 1)

    print(f"Computed difference raster saved to {dif_path}")

In [55]:
def clip_raster_by_vector_bbox_seq(bbox_files,exp_dir,s1_fn,edem_wgs_fn,vrt):
    clip_paths = []
    for bbox_fn in bbox_files:
        tname = bbox_fn.split('/')[-1][:-14] # go back to where you creare this add put __
        print(len(tname))
        if len(tname) > 13:
            tname = tname[:-1]
        print(tname)
        toutdir = join(exp_dir, tname)
        os.makedirs(toutdir, exist_ok=True)
        dsm_path= f"{toutdir}/clipped_edem.tif"
        dtm_path=f"{toutdir}/clipped_ldem.tif"
        dif_path =f"{toutdir}/clipped_zdif.tif"
        s1_path = f"{toutdir}/clipped_s1.tif"


        clip_raster_by_vector_bbox(s1_fn,s1_path , bbox_fn)
        clip_raster_by_vector_bbox(edem_wgs_fn, dsm_path, bbox_fn)
        clip_raster_by_vector_bbox(vrt,dtm_path , bbox_fn)
        try:
            create_zdiff(dsm_path, dtm_path, dif_path, ndv=-9999.0)
            clip_paths.append({'bbox_fn':bbox_fn, 's1':s1_path,'edem':dsm_path,
                            'ldem':dtm_path, 'zdif':dif_path
                        })
        except:
            print('some thing')
    csvfile = join(exp_dir, 'clip_paths.csv')
    dfpaths = pd.DataFrame(clip_paths)
    dfpaths.to_csv(csvfile, index=False)
    return csvfile ,dfpaths

In [56]:
csvfile ,dfpaths = clip_raster_by_vector_bbox_seq(bbox_files,exp_dir,s1_fn,edem_wgs_fn,vrt) # replace bbox_vector by tif

13
DTM_NP_T-0007
Output file /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment/DTM_NP_T-0007/clipped_zdif.tif already exists. Skipping computation.
13
DTM_NP_T-0018
some thing
13
DTM_NP_T-0019
some thing
13
DTM_NP_T-0020
some thing
14
DTM_NP_T-0215
some thing
14
DTM_NP_T-0384
some thing
14
DTM_NP_T-0388
Computed difference raster saved to /media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/experiment/DTM_NP_T-0388/clipped_zdif.tif
14
DTM_NP_T-0389
some thing


In [40]:
dfpaths

,bbox_fn,s1,edem,ldem
0,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...
1,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...
2,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...
3,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...
4,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...
5,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...
6,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...
7,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...,/media/ljp238/12TBWolf/BRCHIEVE/REFERENCE_DEM/...


In [ ]:
clip_raster_by_vector_bbox(s1_fn, f"{toutdir}/clipped_s1.tif", bbox_fn)
clip_raster_by_vector_bbox(edem_wgs_fn, f"{toutdir}/clipped_edem.tif", bbox_fn)
clip_raster_by_vector_bbox(vrt, f"{toutdir}/clipped_ldem.tif", bbox_fn)